In [2]:
import duckdb
import polars as pl
import pandas as pd
import numpy as np
import os
import requests 
import json 
import re
import importlib  
import geopandas as gpd




In [8]:
#where are trips we are missing
# rt pid total trips processed trips


trips_DIR = "../cta-stop-etl/out/cta_bus_full_day_data_v2.parquet"
# combine with duckdb
cmd_number = f"""
SELECT rt, cast(pid as int) as pid, count(distinct unique_trip_vehicle_day) as og_trips
FROM read_parquet('{trips_DIR}')
group by rt, pid
"""

trip_og_pids = duckdb.execute(cmd_number).df()

trips_actual_DIR = "../cta-stop-etl/out/trips/*.parquet"
# combine with duckdb
cmd_number = f"""
SELECT rt, cast(pid as int) as pid, count(distinct unique_trip_vehicle_day) as actual_trips
FROM read_parquet('{trips_actual_DIR}')
group by rt, pid
"""

trip_actual_pids = duckdb.execute(cmd_number).df()

In [22]:
missing_pids = join[join['actual_trips']==0][['rt','pid']].drop_duplicates().sort_values('rt').reset_index(drop=True)

In [25]:
missing_pids[missing_pids['rt']== '55']

,rt,pid
43,55,20424


In [23]:
missing_pids.to_csv('missing_pids.csv', index=False)

In [9]:
join = trip_og_pids.merge(trip_actual_pids, on=['rt', 'pid'], how='left')

join['actual_trips'] = join['actual_trips'].fillna(0)

join['missing_trips_errors'] = np.where(join['actual_trips'] != 0, (join['og_trips'] - join['actual_trips']),0)
join['missing_trips_missing_pids'] = np.where(join['actual_trips'] == 0, (join['og_trips'] - join['actual_trips']),0)


print(f"We at most could add {join['missing_trips_missing_pids'].sum():,} trips which is {join['missing_trips_missing_pids'].sum() / join['og_trips'].sum() * 100:.2f}% of the total trips")

print(join['missing_trips_errors'].sum(), join['missing_trips_errors'].sum() / join['og_trips'].sum() * 100)

#missing_pids.sort_values('missing_trips', ascending=False).head(20)
by_rt = join.groupby('rt').agg({'missing_trips_errors': 'sum','missing_trips_missing_pids': 'sum', 'og_trips': 'sum', 'actual_trips': 'sum', 'pid': lambda x: set(x)})
by_rt['missing_pid_percent'] = (by_rt['missing_trips_missing_pids'] / by_rt['og_trips']) * 100
by_rt['error_percent'] = (by_rt['missing_trips_errors'] / by_rt['og_trips']) *100

by_rt.sort_values('missing_pid_percent', ascending=False).head(20)

We at most could add 74,193.0 trips which is 0.72% of the total trips
280141.0 2.7368253017980493


,missing_trips_errors,missing_trips_missing_pids,og_trips,actual_trips,pid,missing_pid_percent,error_percent
rt,,,,,,,
X98,0.0,193.0,193,0.0,{6672},100.000000,0.000000
124,1500.0,9839.0,54239,42900.0,"{17466, 4708, 10950}",18.140084,2.765538
4,3268.0,14263.0,144759,127228.0,"{16543, 16544, 432, 10928, 19378, 19379, 19380...",9.852928,2.257545
3,4015.0,12774.0,133077,116288.0,"{5348, 18412, 18413, 18414, 18415, 18416, 1841...",9.598954,3.017050
192,263.0,862.0,9226,8101.0,"{6979, 6980, 2671, 19367}",9.343161,2.850639
67,3346.0,9092.0,132681,120243.0,"{4608, 4609, 4610, 4611, 11674, 11675, 11676, ...",6.852526,2.521838
63,4033.0,4352.0,150072,141687.0,"{10882, 4110, 4111, 4112, 4115, 4116, 11682, 1...",2.899941,2.687377
136,101.0,358.0,13248,12789.0,"{4080, 7385, 7386}",2.702295,0.762379
82,2411.0,3566.0,134594,128617.0,"{1664, 6787, 1679, 4625, 8468, 7466, 10925, 35...",2.649449,1.791313


In [131]:
df_pattern_raw = pd.read_parquet('../scrapers/out/gtfs/0f3553898aa5bf17ec6ba588231c97dc7c249f5f_stops.parquet')
df_pattern_raw


,route_id,stop_id,stop_sequence,pid,stop_lat,stop_lon
0,77,17833,40,01526,41.9396756,-87.66589055
1,70,1990,21,01500,41.90259074,-87.72644847
2,90,8535,2,05917,41.98078714,-87.80713047
3,53,9089,2,08232,41.87515999,-87.72537521
4,72,15483,30,10913,41.90974851,-87.72867746
...,...,...,...,...,...,...
40352,66,613,17,04357,41.89651,-87.64817713
40353,21,14845,1,05019,41.85169965,-87.72527203
40354,126,25,23,10903,41.87708065,-87.72082664
40355,9,17063,27,09794,41.86899322,-87.66659907


In [136]:
df_pattern_raw = pd.read_parquet('../cta-stop-etl/out/patterns_current/pid_95_stop.parquet')
df_pattern_raw

,seq,lat,lon,typ,stpid,stpnm,pdist,geometry,segment,p_stp_id
0,1,41.911304,-87.631222,S,923,North Ave & Clark Terminal,0.0,b'\x01\x01\x00\x00\x00H\xa1\xf5\xf0e\xe8U\xc0\...,0,95-923
1,2,41.911241,-87.631290,W,None,None,0.0,"b'\x01\x01\x00\x00\x00\xae\x12,\x0eg\xe8U\xc0\...",1,None
2,3,41.911237,-87.631391,W,None,None,0.0,b'\x01\x01\x00\x00\x00G\xff\xcb\xb5h\xe8U\xc0\...,2,None
3,4,41.911239,-87.631995,W,None,None,0.0,b'\x01\x01\x00\x00\x00\x13\x0f(\x9br\xe8U\xc0}...,3,None
4,5,41.911235,-87.632092,S,924,North Ave & Clark,277.0,b'\x01\x01\x00\x00\x00\x0c\x04\x012t\xe8U\xc0\...,4,95-924
...,...,...,...,...,...,...,...,...,...,...
432,433,41.910720,-87.804430,W,None,None,0.0,b'\x01\x01\x00\x00\x00\xf6z\xf7\xc7{\xf3U\xc0\...,432,None
433,434,41.910730,-87.804000,W,None,None,0.0,b'\x01\x01\x00\x00\x00\xfa~j\xbct\xf3U\xc08\xb...,433,None
434,435,41.909400,-87.803950,W,None,None,0.0,b'\x01\x01\x00\x00\x00\xa1g\xb3\xeas\xf3U\xc0\...,434,None
435,436,41.909284,-87.804012,W,None,None,0.0,b'\x01\x01\x00\x00\x00\xdce\xbf\xeet\xf3U\xc0V...,435,None


In [102]:
df_pattern_raw = pd.read_parquet('../cta-stop-etl/out/patterns_raw/pid_95_raw.parquet')
df_pattern_raw

,seq,lat,lon,typ,stpid,stpnm,pdist
0,1,41.911304,-87.631222,S,923,North Ave & Clark Terminal,0.0
1,2,41.911241,-87.631290,W,None,None,0.0
2,3,41.911237,-87.631391,W,None,None,0.0
3,4,41.911239,-87.631995,W,None,None,0.0
4,5,41.911235,-87.632092,S,924,North Ave & Clark,277.0
...,...,...,...,...,...,...,...
432,433,41.910720,-87.804430,W,None,None,0.0
433,434,41.910730,-87.804000,W,None,None,0.0
434,435,41.909400,-87.803950,W,None,None,0.0
435,436,41.909284,-87.804012,W,None,None,0.0


In [3]:
# look at test trip df
test_actual = pl.read_parquet("../cta-stop-etl/out/trips/trips_1009_full.parquet")
test_actual.head()

seg_combined,typ,stop_sequence,bus_stop_time,stop_dist,speed_mph,unique_trip_vehicle_day,stpid,p_stp_id,vid,rt,pid
f64,str,i64,datetime[ns],f64,f64,str,str,str,str,str,str
0.0,"""S""",1,2023-04-11 07:00:05.842126106,327.622878,6.419818,"""241009.0104335911962023-04-11""","""6439""","""1009-6439""","""1196""","""24""","""1009.0"""
3.0,"""S""",2,2023-04-11 07:03:37.671807211,218.569266,6.419818,"""241009.0104335911962023-04-11""","""6440""","""1009-6440""","""1196""","""24""","""1009.0"""
6.0,"""S""",3,2023-04-11 07:05:15.386392829,208.038412,6.419818,"""241009.0104335911962023-04-11""","""6441""","""1009-6441""","""1196""","""24""","""1009.0"""
8.0,"""S""",4,2023-04-11 07:06:26.318364889,228.6532,6.419818,"""241009.0104335911962023-04-11""","""14538""","""1009-14538""","""1196""","""24""","""1009.0"""
11.0,"""S""",5,2023-04-11 07:06:59.933675321,305.68342,6.419818,"""241009.0104335911962023-04-11""","""6443""","""1009-6443""","""1196""","""24""","""1009.0"""


In [15]:
# look at test og df
test_actual = pl.read_parquet("../cta-stop-etl/out/pids/1009.parquet")
test_actual.head()

vid,tmstmp,lat,lon,hdg,pid,rt,des,pdist,dly,tatripid,tablockid,zone,scrape_file,data_time,data_hour,data_date,unique_trip_vehicle_day
u32,str,f64,f64,u32,f64,str,str,str,bool,str,str,str,str,str,str,str,str
7940,"""20220822 07:02""",41.752548,-87.633652,13,1009.0,"""24""","""59th""","""0""",false,"""1043359""","""3 -721""",null,"""bus_data/2022-08-22/07:02:56.j…","""2022-08-22 07:02:00""","""7""","""2022-08-22""","""241009.0104335979402022-08-22"""
7940,"""20220822 07:07""",41.759369,-87.630676,19,1009.0,"""24""","""59th""","""2703""",false,"""1043359""","""3 -721""",null,"""bus_data/2022-08-22/07:07:57.j…","""2022-08-22 07:07:00""","""7""","""2022-08-22""","""241009.0104335979402022-08-22"""
7940,"""20220822 07:12""",41.777223,-87.630043,0,1009.0,"""24""","""59th""","""9277""",false,"""1043359""","""3 -721""",null,"""bus_data/2022-08-22/07:12:56.j…","""2022-08-22 07:12:00""","""7""","""2022-08-22""","""241009.0104335979402022-08-22"""
1370,"""20220823 07:02""",41.752088,-87.633785,11,1009.0,"""24""","""59th""","""0.0""",false,"""1043359""","""3 -721""",null,"""bus_data/2022-08-23/07:02:56.j…","""2022-08-23 07:02:00""","""7""","""2022-08-23""","""241009.0104335913702022-08-23"""
1370,"""20220823 07:07""",41.761951,-87.629601,1,1009.0,"""24""","""59th""","""3679.0""",false,"""1043359""","""3 -721""",null,"""bus_data/2022-08-23/07:07:56.j…","""2022-08-23 07:07:00""","""7""","""2022-08-23""","""241009.0104335913702022-08-23"""


In [137]:
# counts of trips by pattern, processed

trips_actual_DIR = "../cta-stop-etl/out/trips/*.parquet"

# combine with duckdb
cmd_pid_ = f"""
SELECT pid, count( distinct unique_trip_vehicle_day)
FROM read_parquet('{trips_actual_DIR}')
group by pid
order by count( distinct unique_trip_vehicle_day) desc
"""

trips_by_pid = duckdb.execute(cmd_pid_).df()

In [23]:
# counts of trips by pattern in OG data

trips_og_DIR = "../cta-stop-etl/out/pids/*.parquet"

# combine with duckdb
cmd_pid_ = f"""
SELECT pid, count( distinct unique_trip_vehicle_day)
FROM read_parquet('{trips_og_DIR}')
group by pid
order by count( distinct unique_trip_vehicle_day) desc
"""

trips_by_pid_og = duckdb.execute(cmd_pid_).df()

In [59]:
pids_in_raw = []
raw_patterns_DIR = "../cta-stop-etl/out/patterns_raw/"
for pid_file in os.listdir(raw_patterns_DIR):
# Skip auxiliary files in folder, only process parquets
    if not pid_file.endswith(".parquet"):
        continue
    numbers = re.findall(r"\d+", pid_file)
    pid = numbers[0]
    pids_in_raw.append(pid)

pids_in_current_processed = []
raw_patterns_DIR = "../cta-stop-etl/out/patterns_current/"
for pid_file in os.listdir(raw_patterns_DIR):
# Skip auxiliary files in folder, only process parquets
    if not pid_file.endswith(".parquet"):
        continue
    numbers = re.findall(r"\d+", pid_file)
    pid = numbers[0]
    pids_in_current_processed.append(pid)

pids_in_current_processed= set(pids_in_current_processed)

In [16]:
#all patterns in historic
#process_historic_gtfs()

# joins for historic need to be special

DIR_INP = "../scrapers/out/gtfs/"

historic_gtfs_shapes = os.listdir(DIR_INP)

all_historic_pids = set()
for snapshot in historic_gtfs_shapes:

    # Check that file is valid (a parquet file)
    if not snapshot.endswith(".parquet"):
        continue

    print(f"\tChecking PIDs in {snapshot}")

    df_gtfs_patterns = pd.read_parquet(DIR_INP + '/' + snapshot)

    # Get unique pids
    snapshot_pids = set(df_gtfs_patterns["shape_id"].unique())
    print(snapshot_pids)

    all_historic_pids =  all_historic_pids | snapshot_pids



	Checking PIDs in f95f1ef9f77df33dd297e021382c7fe77d8695d2.parquet
{'65400749', '65300420', '65410956', '65401039', '307100033', '65408142', '65400758', '307100012', '307200012', '307200011', '307200009', '307200053', '65400712', '307100001', '307100074', '307100035', '65301039', '307200001', '65300758', '65404369', '65400703', '307100053', '307100011', '65404493', '307200010', '65400710', '65300730', '65400711', '65304335', '65304680', '65306357', '65300703', '65300729', '65310956', '65400757', '65304369', '307200074', '65406357', '65400730', '65410934', '65404335', '307100010', '65301275', '307100002', '65308142', '307200002', '65402673', '307100009', '307200033', '65300712', '65300757', '65302673', '65304492', '65310934', '65404680', '307200035', '65401275', '65300711', '65306354', '65304493', '65300749', '65406354', '65404492', '65300710', '65400420', '65400729'}
	Checking PIDs in d88906cf3efa4646a64cf416e960d05e146c4cbc.parquet
{'308000053', '308000011', '66000730', '66001275', '6

In [46]:
pids_in_hist = []
hist_patterns_DIR = "../cta-stop-etl/out/patterns_historic/"
for pid_file in os.listdir(hist_patterns_DIR):
# Skip auxiliary files in folder, only process parquets
    if not pid_file.endswith(".parquet"):
        continue
    numbers = re.findall(r"\d+", pid_file)
    pid = numbers[0]
    pids_in_hist.append(pid)

In [24]:
pid_in_trips = trips_by_pid['pid'].astype(float).astype(int).astype(str).tolist()
pid_in_og = trips_by_pid_og['pid'].astype(float).astype(int).astype(str).tolist()
# pids_in_raw
# pids_in_hist

In [36]:
# Current patterns from the API only give me 715 patterns
# before we had 843 

In [54]:
#make all setse
pid_in_trips = set(pid_in_trips)
pid_in_og = set(pid_in_og)
pids_in_raw = set(pids_in_raw)
pids_in_historic = set(pids_in_hist)
pids_in_current_processed= set(pids_in_current_processed)


# how many patterns do we have from the raw data
print(f"There are {len(pid_in_og)} patterns in the raw bus ping data.")
len(pid_in_og.intersection(pids_in_raw.union(pids_in_historic)))

845

In [91]:
def build_merged_pattern_data(
    df_shapes: pl.DataFrame, df_stops: pl.DataFrame, df_trips: pl.DataFrame
) -> pl.DataFrame:
    """
    Takes data frames with GTFS data and reproduces the CTA API "pattern"
    tables in a standardized data frame for further cleaning.

    Input:
        - df_shapes: Data frame with GTFS shapes info
        - df_stops: Data frame with GTFS stops info
        - df_trips: Data frame with GTFS trips info

    Outputs:
        - A data frame with "shape", "stops" and "trip" data in a format
        that resembles the pattern data from the CTA API. A single file is
        produced for all the pids.
    """

    # Preprocess GTFS shapes file to have the same format as the CTA patterns
    df_shapes = df_shapes.rename(
        {
            "shape_pt_lat": "lat",
            "shape_pt_lon": "lon",
            "shape_pt_sequence": "seq",
            "shape_dist_traveled": "pdist",
        }
    )

    #add index
    df_shapes = df_shapes.with_row_index("shape_index")


    # convert to geodataframe
    gdf_shapes = gpd.GeoDataFrame(
        df_shapes.to_pandas(),
        geometry=gpd.points_from_xy(df_shapes.to_pandas()["lon"], df_shapes.to_pandas()["lat"]),
        crs="EPSG:4326",
    )
    # for distance calculations
    gdf_shapes = gdf_shapes.to_crs(26971)

    # Preprocess GTFS stops file to have the same format as the CTA patterns
    df_stops = df_stops[["stop_id", "stop_name", "stop_lat", "stop_lon", "location_type", "parent_station", 
                         'stop_desc']]
    df_stops = df_stops.rename(
        {"stop_id": "stpid", "stop_name": "stpnm", "stop_lat": "lat", "stop_lon": "lon"}
    )

    # just bus stops
    df_stops = df_stops.filter((pl.col('location_type') == '0') & (pl.col('parent_station').is_null()))
    
    df_stops = df_stops.with_columns(pl.lit("S").alias("typ"))


    # convert to geodataframe
    gdf_stops = gpd.GeoDataFrame(
        df_stops.to_pandas(),
        geometry=gpd.points_from_xy(df_stops.to_pandas()["lon"], df_stops.to_pandas()["lat"]),
        crs="EPSG:4326",
    )

    # for distance calculations
    gdf_stops= gdf_stops.to_crs(26971)


    # Recreate the CTA API "patterns" file by merging GTFS shapes and stops
    gdf_patterns = gpd.sjoin_nearest(
        gdf_shapes, gdf_stops, how="left", max_distance=5, distance_col="dist"
    )

    return gdf_patterns

In [104]:
from zipfile import ZipFile, BadZipFile
path = "../scrapers/inp/historic_gtfs/0f3553898aa5bf17ec6ba588231c97dc7c249f5f.zip"
with ZipFile(path) as gtfs_zip:
        with gtfs_zip.open("shapes.txt") as shapes:
                df_shapes = pl.read_csv(shapes, infer_schema_length=0)
        with gtfs_zip.open("stops.txt") as stops:
                df_stops = pl.read_csv(stops, infer_schema_length=0)
        with gtfs_zip.open("trips.txt") as trips:
                df_trips = pl.read_csv(trips,infer_schema_length=0)
        
        with gtfs_zip.open("stop_times.txt") as stop_times:
            df_stop_times = pl.read_csv(stop_times, infer_schema_length=0)



/var/folders/vk/_03tl7qs1sv80_pgkhd8bxch0000gn/T/ipykernel_10234/1774231597.py:5: UserWarning: Polars found a filename. Ensure you pass a path to the file instead of a python file object when possible for best performance.
  df_shapes = pl.read_csv(shapes, infer_schema_length=0)
/var/folders/vk/_03tl7qs1sv80_pgkhd8bxch0000gn/T/ipykernel_10234/1774231597.py:7: UserWarning: Polars found a filename. Ensure you pass a path to the file instead of a python file object when possible for best performance.
  df_stops = pl.read_csv(stops, infer_schema_length=0)
/var/folders/vk/_03tl7qs1sv80_pgkhd8bxch0000gn/T/ipykernel_10234/1774231597.py:9: UserWarning: Polars found a filename. Ensure you pass a path to the file instead of a python file object when possible for best performance.
  df_trips = pl.read_csv(trips,infer_schema_length=0)
/var/folders/vk/_03tl7qs1sv80_pgkhd8bxch0000gn/T/ipykernel_10234/1774231597.py:12: UserWarning: Polars found a filename. Ensure you pass a path to the file instead o

In [110]:
df_stops

stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,location_type,parent_station,wheelchair_boarding
str,str,str,str,str,str,str,str,str
"""1""","""1""","""Jackson & Austin Terminal""","""Jackson & Austin Terminal, Nor…","""41.87632184""","""-87.77410482""","""0""",null,"""1"""
"""2""","""2""","""5900 W Jackson""","""5900 W Jackson, Eastbound, Sou…","""41.87706679""","""-87.77131794""","""0""",null,"""1"""
"""4""","""4""","""5700 W Jackson""","""5700 W Jackson, Eastbound, Sou…","""41.87702418""","""-87.76745055""","""0""",null,"""1"""
"""6""","""6""","""Jackson & Lotus""","""Jackson & Lotus, Eastbound, So…","""41.876513""","""-87.761446""","""0""",null,"""1"""
"""7""","""7""","""5351 W Jackson""","""5351 W Jackson, Eastbound, Sou…","""41.87655197""","""-87.75892544""","""0""",null,"""1"""
…,…,…,…,…,…,…,…,…
"""60009""",null,"""Sox-35th (33rd St entry)""","""Auxiliary entrance farecard re…","""41.83458""","""-87.6307""","""2""","""40190""","""2"""
"""60010""",null,"""Sox-35th (35th St entry)""","""Main entrance accessible""","""41.83106""","""-87.6307""","""2""","""40190""","""1"""
"""60011""",null,"""Wilson (Wilson Av south side e…","""Main entrance on south side of…","""41.96528""","""-87.6579""","""2""","""40540""","""1"""


In [118]:
join1 = df_trips.join(df_stop_times, on="trip_id", coalesce=True)

stops_pids = join1.select(['route_id','shape_id', 'stop_id','stop_sequence' ]).unique()

stops_pids = stops_pids.with_columns(pl.col("shape_id").cast(pl.String))
stops_pids = stops_pids.with_columns(pl.col("shape_id").str.slice(-5).cast(pl.Float32).cast(pl.Int32).cast(pl.String).alias("pid"))

stops_pids = stops_pids.drop("shape_id")

stop_geom = df_stops.select(['stop_id','stop_lat','stop_lon'])

#join geometry
stops_pids.join(stop_geom, on="stop_id", how="left", coalesce=True)


route_id,stop_id,stop_sequence,pid,stop_lat,stop_lon
str,str,str,str,str,str
"""81W""","""8581""","""13""","""6547""","""41.96628915""","""-87.84606577"""
"""54""","""10284""","""13""","""4587""","""41.87036117""","""-87.74479614"""
"""93""","""11901""","""15""","""1800""","""41.983033""","""-87.699079"""
"""4""","""1565""","""85""","""19383""","""41.841882""","""-87.62324"""
"""62H""","""7703""","""8""","""6137""","""41.779022""","""-87.800827"""
…,…,…,…,…,…
"""65""","""16028""","""6""","""7908""","""41.89177539""","""-87.6256301"""
"""30""","""17559""","""11""","""8228""","""41.66589253""","""-87.53916093"""
"""80""","""5676""","""2""","""10811""","""41.95455981""","""-87.65434308"""


In [92]:
test = build_merged_pattern_data(df_shapes, df_stops, df_trips)

In [80]:
test[test['dist'].notna()]

,shape_index,shape_id,lat_left,lon_left,seq,pdist,geometry,index_right,stpid,stpnm,lat_right,lon_right,location_type,parent_station,typ,dist
0,0,66200095,41.911304,-87.631222,1,0,POINT (358249.873 582489.585),570.0,923,North Ave & Clark Terminal,41.91128,-87.631195,0,None,S,3.482010
1,1,66200095,41.911304,-87.631222,2,13,POINT (358249.873 582489.585),570.0,923,North Ave & Clark Terminal,41.91128,-87.631195,0,None,S,3.482010
2,2,66200095,41.91127,-87.631234,3,24,POINT (358248.908 582485.801),570.0,923,North Ave & Clark Terminal,41.91128,-87.631195,0,None,S,3.421053
8,8,66200095,41.911237,-87.631988,9,280,POINT (358186.383 582481.623),571.0,924,North Ave & Clark,41.9112345,-87.63202175,0,None,S,2.813877
92,92,66200095,41.911031,-87.645686,93,4001,POINT (357050.120 582449.541),9681.0,16142,North Ave & Orchard,41.91102555,-87.64571437,0,None,S,2.430379
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
640825,640825,308200008,41.90387019,-87.62874489,177,67071,POINT (358462.167 581665.600),3571.0,5371,Division & State,41.90387512,-87.62874921,0,None,S,0.654484
644793,644793,308200053,41.87552181,-87.64710038,2,692,POINT (356964.321 578504.620),10380.0,18184,UIC-Halsted Blue Line Station,41.87550381,-87.6470767,0,None,S,2.803821
645524,645524,308200074,41.87552181,-87.64710038,366,100121,POINT (356964.321 578504.620),10380.0,18184,UIC-Halsted Blue Line Station,41.87550381,-87.6470767,0,None,S,2.803821
645698,645698,308200128,41.87552181,-87.64710038,15,3372,POINT (356964.321 578504.620),10380.0,18184,UIC-Halsted Blue Line Station,41.87550381,-87.6470767,0,None,S,2.803821


In [97]:
#should only be one stpid, pick the closest point from the shape file
test["rn"] = test.groupby(['shape_id',"stpid"])["dist"].rank(method="first", ascending=True)


test_dedupe = test[(test['rn'] == 1) | (test['rn'].isna())]

#here are two difference sequences that have a dist between but same lat and lon. i think its a precision issue
# not sure if this will be an issue
test[test["rn"] > 1].sort_values(['shape_id',"stpid"]).head(50)

,shape_index,shape_id,lat_left,lon_left,seq,pdist,geometry,index_right,stpid,stpnm,lat_right,lon_right,location_type,parent_station,stop_desc,typ,dist,rn
637674,637674,308100035,41.85413059,-87.70542796,281,86246,POINT (352139.893 576091.661),2031.0,3055,Kedzie Pink Line Station,41.85409804,-87.70539304,0,None,"Kedzie Pink Line Station, Southbound, Northwes...",S,4.634609,2.0
644596,644596,308200035,41.85413059,-87.70542796,281,86246,POINT (352139.893 576091.661),2031.0,3055,Kedzie Pink Line Station,41.85409804,-87.70539304,0,None,"Kedzie Pink Line Station, Southbound, Northwes...",S,4.634609,2.0
0,0,66200095,41.911304,-87.631222,1,0,POINT (358249.873 582489.585),570.0,923,North Ave & Clark Terminal,41.91128,-87.631195,0,None,"North Ave & Clark Terminal, Southwestbound, Bu...",S,3.482010,2.0
1,1,66200095,41.911304,-87.631222,2,13,POINT (358249.873 582489.585),570.0,923,North Ave & Clark Terminal,41.91128,-87.631195,0,None,"North Ave & Clark Terminal, Southwestbound, Bu...",S,3.482010,3.0
384,384,66200095,41.91034,-87.68951,385,15942,POINT (353414.855 582344.572),586.0,950,North Ave & Campbell,41.91036749,-87.68953036,0,None,"North Ave & Campbell, Westbound, Northeast Corner",S,3.489506,2.0
449,449,66200095,41.91027,-87.69893,450,18451,POINT (352633.378 582330.973),588.0,954,North Ave & Francisco,41.91030323,-87.69890875,0,None,"North Ave & Francisco, Westbound, Northeast Co...",S,4.090409,2.0
451,451,66200095,41.91027,-87.69893,452,18492,POINT (352633.378 582330.973),588.0,954,North Ave & Francisco,41.91030323,-87.69890875,0,None,"North Ave & Francisco, Westbound, Northeast Co...",S,4.090409,3.0
685,685,66200095,41.90981,-87.73678,686,28734,POINT (349493.489 582257.348),597.0,971,North Ave & Kostner/Grand,41.909845,-87.736802,0,None,"North Ave & Kostner/Grand, Westbound, Northwes...",S,4.294712,2.0
687,687,66200095,41.90981,-87.73678,688,28780,POINT (349493.489 582257.348),597.0,971,North Ave & Kostner/Grand,41.909845,-87.736802,0,None,"North Ave & Kostner/Grand, Westbound, Northwes...",S,4.294712,3.0
833,833,66200095,41.90949,-87.76041,834,35176,POINT (347533.239 582208.441),603.0,981,North Ave & Long,41.90950919,-87.76039976,0,None,"North Ave & Long, Westbound, Northeast Corner",S,2.294552,2.0


In [95]:
#test for dupe shape_index
test_dedupe.groupby('shape_index').size()[test_dedupe.groupby('shape_index').size() > 1]

shape_index
100721    2
104180    2
149660    2
250641    2
251946    2
254684    2
320536    2
338347    2
341231    2
481613    2
517155    2
586286    2
dtype: int64

In [96]:
test_dedupe[test_dedupe['shape_index'] == 100721]

,shape_index,shape_id,lat_left,lon_left,seq,pdist,geometry,index_right,stpid,stpnm,lat_right,lon_right,location_type,parent_station,stop_desc,typ,dist,rn
100721,100721,66201659,41.939058,-87.712582,1,0,POINT (351477.586 585520.162),10510.0,18394,Kimball & Belmont (Blue Line),41.9390928,-87.71256509,0,None,,S,4.111865,1.0
100721,100721,66201659,41.939058,-87.712582,1,0,POINT (351477.586 585520.162),7052.0,11139,Kimball & Belmont (Blue Line),41.9390928,-87.71256509,0,None,"Kimball & Belmont (Blue Line), Southbound, Sou...",S,4.111865,1.0


In [87]:
df_stops.head()

stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,location_type,parent_station,wheelchair_boarding
str,str,str,str,str,str,str,str,str
"""1""","""1""","""Jackson & Austin Terminal""","""Jackson & Austin Terminal, Nor…","""41.87632184""","""-87.77410482""","""0""",null,"""1"""
"""2""","""2""","""5900 W Jackson""","""5900 W Jackson, Eastbound, Sou…","""41.87706679""","""-87.77131794""","""0""",null,"""1"""
"""4""","""4""","""5700 W Jackson""","""5700 W Jackson, Eastbound, Sou…","""41.87702418""","""-87.76745055""","""0""",null,"""1"""
"""6""","""6""","""Jackson & Lotus""","""Jackson & Lotus, Eastbound, So…","""41.876513""","""-87.761446""","""0""",null,"""1"""
"""7""","""7""","""5351 W Jackson""","""5351 W Jackson, Eastbound, Sou…","""41.87655197""","""-87.75892544""","""0""",null,"""1"""


In [86]:
# has every bus stop been assigned to a shape_index?
all_bus_stops = set(df_stops['stpid'].unique())

all_bus_stops_assigned = set(test_dedupe['stpid'].unique())

ColumnNotFoundError: stpid

{'5371',
 '7399',
 '8100',
 '2979',
 '8299',
 '9572',
 '1504',
 '11896',
 '15496',
 '1043',
 '3182',
 '6638',
 '8189',
 '2981',
 '1329',
 '950',
 '5395',
 '6478',
 '11092',
 '7276',
 '1367',
 '6959',
 '794',
 '15989',
 '11320',
 '9195',
 '7795',
 '9605',
 '12069',
 '4570',
 '5456',
 '5973',
 '15804',
 '1139',
 '796',
 '9502',
 '1450',
 '10898',
 '5547',
 '1297',
 '2955',
 '10029',
 '2919',
 '12256',
 '3256',
 '17550',
 '4591',
 '4863',
 '2501',
 '12666',
 '11137',
 '2246',
 '2751',
 '5727',
 '18507',
 '4299',
 '3338',
 '9496',
 '4571',
 '2392',
 '12137',
 '2922',
 '3407',
 '12149',
 '776',
 '15826',
 '7196',
 '4125',
 '1580',
 '9492',
 '5924',
 '7759',
 '17186',
 '1616',
 '17345',
 '16114',
 '10102',
 '11519',
 '12613',
 '12372',
 '18569',
 '10342',
 '12163',
 '17719',
 '10565',
 '9178',
 '1047',
 '6311',
 '12152',
 '7494',
 '56',
 '18288',
 '11150',
 '11287',
 '4484',
 '2161',
 '3547',
 '17729',
 '5187',
 '6066',
 '3717',
 '10402',
 '9522',
 '3667',
 '17739',
 '9493',
 '7275',
 '3449'

In [ ]:
test[test["rn"] > 1].sort_values(['shape_id',"stpid"]).head(50)